<a href="https://colab.research.google.com/github/dzikri200/sistem_rekomendasi/blob/main/Sistem_rekomendasi_0070.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import re
import random

In [5]:
df = pd.read_csv('dataset.csv')
df.head()

,Name,Addres,Modul
0,Dzikri,Demak,Text Minning Machine Learning Jejaring Sosial ...
1,Dicoding,Bandung,"Refleksi diri, Belajar Dasar Git dengan Github..."
2,Binar Academy,Kab. Tangerang,"Mampu mendeklarasi Dasar-dasar Activity, Fragm..."
3,Hacktiv8,Jakarta Selatan,"Android Java for Mobile Developer, Memahami se..."
4,Zenius,Kota Jakarta Selatan,"Data Analytics, Dasar-Dasar Data Science, Pyth..."


In [6]:
df.describe()

,Name,Addres,Modul
count,11,11,11
unique,11,8,11
top,Dzikri,Kota Jakarta Selatan,Text Minning Machine Learning Jejaring Sosial ...
freq,1,4,1


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Name    11 non-null     object
 1   Addres  11 non-null     object
 2   Modul   11 non-null     object
dtypes: object(3)
memory usage: 392.0+ bytes


In [12]:
def print_description(index):
    example = df[df.index == index][['Name', 'Addres','Modul']].values[0]
    if len(example) > 0:
        print(example[0])
        print('Name :', example[1])
        print('Addres:', example[2])

In [13]:
print_description(0)

Dzikri
Name : Demak
Addres: Text Minning Machine Learning Jejaring Sosial Sistem Informasi Geografis Grafika Komputer Basis Data Terdistribusi Web Service


In [14]:
print_description(5)

BISA AI ACADEMY 
Name : Kota Jakarta Selatan
Addres: Bisnis Kecerdasan Artifisial (AI-HUSTLER), Jr. Project Manager, Mengenal teknik dan langkah dalam manajemen proyek ,Mengenal teknik dan langkah dalam manajemen proyek perangkat lunak


In [15]:
print_description(10)

NF Computer PT Nurul Fikri Cipta Inovasi
Name : Jawa Barat
Addres: Soft Skill UI UX Design Pattern Pemrograman Mobile Code Versioning Dasar Dasar Pemrograman Databases Final Project


In [17]:
import nltk
nltk.download('stopwords')
clean_spcl = re.compile('[/(){}\[\]\|@,;]')
clean_symbol = re.compile('[^0-9a-z #+_]')
#stopworda = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = clean_spcl.sub(' ', text)
    text = clean_symbol.sub('', text)
    #text = ' '.join(word for word in text.split() if word not in stopworda) # hapus stopword dari kolom deskripsi
    return text
    
df['modul_clean'] = df['Modul'].apply(clean_text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [18]:
df.head()

,Name,Addres,Modul,modul_clean
0,Dzikri,Demak,Text Minning Machine Learning Jejaring Sosial ...,text minning machine learning jejaring sosial ...
1,Dicoding,Bandung,"Refleksi diri, Belajar Dasar Git dengan Github...",refleksi diri belajar dasar git dengan github...
2,Binar Academy,Kab. Tangerang,"Mampu mendeklarasi Dasar-dasar Activity, Fragm...",mampu mendeklarasi dasardasar activity fragme...
3,Hacktiv8,Jakarta Selatan,"Android Java for Mobile Developer, Memahami se...",android java for mobile developer memahami se...
4,Zenius,Kota Jakarta Selatan,"Data Analytics, Dasar-Dasar Data Science, Pyth...",data analytics dasardasar data science pytho...


In [19]:
# Deskripsi kedua (Setelah preprocessing)
def print_description_clean(index):
    example = df[df.index == index][['modul_clean', 'Name','Addres']].values[0]
    if len(example) > 0:
        print(example[0])
        print('Name:', example[1])
        print('Addres:', example[2])

In [21]:
print_description_clean(1)

refleksi diri  belajar dasar git dengan github  inisiatif  proaktif  bertanggung jawab  memulai dasar pemrograman untuk menjadi pengembang software
Name: Dicoding
Addres: Bandung


In [22]:
print_description_clean(3)

android java for mobile developer  memahami serta mengimplementasikan library hingga user interface 3rd parties sebagai kerangka kerja awal pembuatan sebuah aplikasi android
Name: Hacktiv8 
Addres: Jakarta Selatan


In [23]:
print_description(10)

NF Computer PT Nurul Fikri Cipta Inovasi
Name : Jawa Barat
Addres: Soft Skill UI UX Design Pattern Pemrograman Mobile Code Versioning Dasar Dasar Pemrograman Databases Final Project


In [25]:
df.set_index('Name', inplace=True)
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(df['modul_clean'])
cos_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
cos_sim

array([[1.        , 0.        , 0.        , 0.        , 0.06910311,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.01894346, 0.        , 0.        ,
        0.        , 0.03038339, 0.        , 0.        , 0.        ,
        0.09305197],
       [0.        , 0.01894346, 1.        , 0.03487117, 0.03503509,
        0.0446118 , 0.        , 0.        , 0.03019265, 0.        ,
        0.01997527],
       [0.        , 0.        , 0.03487117, 1.        , 0.        ,
        0.        , 0.        , 0.        , 0.04929113, 0.01303293,
        0.01405473],
       [0.06910311, 0.        , 0.03503509, 0.        , 1.        ,
        0.01799497, 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.0446118 , 0.        , 0.01799497,
        1.        , 0.02172905, 0.        , 0.        , 0.        ,
        0.01092069],
       [0.        , 0.03038339, 0.        , 0.        , 0.

In [26]:
# Set index utama di kolom 'name'
indices = pd.Series(df.index)
indices[:11]

0                                       Dzikri
1                                     Dicoding
2                               Binar Academy 
3                                    Hacktiv8 
4                                      Zenius 
5                             BISA AI ACADEMY 
6                                    Braindevs
7                          M-Knows Consulting 
8                       Home Credit Indonesia 
9                                 Bukit Vista 
10    NF Computer PT Nurul Fikri Cipta Inovasi
Name: Name, dtype: object

In [27]:
def recommendations(name, cos_sim = cos_sim):
    
    recommended_kpsmerdeka = []
    
    # Mengambil nama hotel berdasarkan variabel indicies
    idx = indices[indices == name].index[0]

    # Membuat series berdasarkan skor kesamaan
    score_series = pd.Series(cos_sim[idx]).sort_values(ascending = False)

    # mengambil index dan dibuat 10 baris rekomendasi terbaik
    top_10_indexes = list(score_series.iloc[1:4].index)
    
    for i in top_10_indexes:
        recommended_kpsmerdeka.append(list(df.index)[i])
        
    return recommended_kpsmerdeka

In [28]:
recommendations('Dzikri')

['Zenius ', 'Dicoding', 'Binar Academy ']